In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Define your database credentials
USERNAME = 'admin'
PASSWORD = 'isba_4715'
HOST = 'isba-dev-01.ctqaesoyaych.us-east-1.rds.amazonaws.com'
PORT = '3306'
DATABASE = 'SQL_Project_Spring_2024'

# Create the database URL for SQLAlchemy
DATABASE_URL = f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

# Create a database engine
engine = create_engine(DATABASE_URL)

In [ ]:
# What are the average premium, deductible, and maximum out-of-pocket (MOOP) 
#costs by plan type and year, and how have these changed over time?

# This query can give us an overview of how insurance costs and coverage options 
# have changed over the years, possibly reflecting changes in market conditions or policy.



In [ ]:
descriptive_query = """
WITH PlanCosts AS (
    SELECT 
        p.plan_year,
        p.plan_type,
        AVG(p.premium) AS avg_premium,
        AVG(d.amount) AS avg_deductible,
        AVG(m.amount) AS avg_moop
    FROM Plans p
    JOIN Deductibles d ON p.db_id = d.db_id
    JOIN MOOPs m ON p.db_id = m.db_id
    GROUP BY p.plan_year, p.plan_type
)

SELECT
    plan_year,
    plan_type,
    avg_premium,
    avg_deductible,
    avg_moop,
    LAG(avg_premium, 1) OVER (PARTITION BY plan_type ORDER BY plan_year) AS prev_year_premium,
    LAG(avg_deductible, 1) OVER (PARTITION BY plan_type ORDER BY plan_year) AS prev_year_deductible,
    LAG(avg_moop, 1) OVER (PARTITION BY plan_type ORDER BY plan_year) AS prev_year_moop
FROM PlanCosts
ORDER BY plan_year, plan_type;
"""

# Execute and display results for Descriptive Analytics
descriptive_results = pd.read_sql(descriptive_query, con=engine)
print("Descriptive Analytics Results:")
print(descriptive_results)



In [ ]:
Insight:
    HMO Plans:
        
            The average premium for HMO plans generally decreased from 2019 to 2020 but 
            showed a fluctuating increase from 2021 onwards.
            
            The average deductible for HMO plans has increased significantly over the 
            years, peaking in 2021 before slightly decreasing.
            
            The average MOOP has also increased steadily over the years, indicating 
            rising potential maximum costs for consumers in these plans.
            
    PPO Plans:
        
            The data for PPO plans is sparse but shows a significant increase in average 
            premiums from 2016 to 2022.
            
            The deductible in PPO plans decreased sharply in 2022 compared to 2016.
            
            MOOPs in PPO plans increased substantially in 2022 compared to earlier data.
            
Recommendation:
    
    For Consumers:
        
            HMO Plan Holders: With increasing deductibles and MOOP, consumers should 
            prepare for potentially higher out-of-pocket expenses, especially in medical 
            scenarios requiring extensive care.
            
            PPO Plan Holders: Given the higher premiums but lower deductibles in 2022, 
            consumers who prefer more flexibility in choosing healthcare providers might 
            consider these plans but should be aware of the higher upfront costs.
            
    For Healthcare Policy Makers and Insurance Providers:
        
            Evaluate the factors contributing to the rise in costs, particularly the 
            steady increase in MOOP, and consider measures to cap these increases to make 
            healthcare more affordable.
            
            Offer more diversified plan options or adjust existing plans' benefits to 
            balance premiums and out-of-pocket costs, catering to consumer affordability
            
Prediction:
    
    Future Trends: 
        
            Given the trend data, it's likely that HMO premiums may continue to 
            fluctuate while deductibles and MOOPs may stabilize or slightly decrease 
            after reaching a peak, considering potential consumer resistance and 
            regulatory actions.
    
    PPO premiums and costs: 
        
            As the market adjusts, PPO premiums might stabilize, but deductibles 
            and MOOPs could increase if the trend towards higher cost sharing continues.

In [ ]:
# Why do some plan types have higher premiums and MOOPs? Identify the correlation between premium costs and deductible/MOOP 
# values within each plan type and year.

# This query explores the relationship between the cost of premiums and the deductible 
# amounts, which can indicate how much consumers are paying upfront versus at the point 
# of service.

In [ ]:

diagnostic_query = """
WITH CostCorrelation AS (
    SELECT
        p.plan_year,
        p.plan_type,
        AVG(p.premium) AS avg_premium,
        AVG(d.amount) AS avg_deductible,
        AVG(m.amount) AS avg_moop,
        COUNT(*) AS n,
        SUM(p.premium * d.amount) AS sum_premium_deductible,
        SUM(p.premium * m.amount) AS sum_premium_moop,
        SUM(p.premium * p.premium) AS sum_premium_squared,
        SUM(d.amount * d.amount) AS sum_deductible_squared,
        SUM(m.amount * m.amount) AS sum_moop_squared
    FROM Plans p
    JOIN Deductibles d ON p.db_id = d.db_id
    JOIN MOOPs m ON p.db_id = m.db_id
    GROUP BY p.plan_year, p.plan_type
)

SELECT
    plan_year,
    plan_type,
    avg_premium,
    avg_deductible,
    avg_moop,
    ((n * sum_premium_deductible - n * avg_premium * avg_deductible) / 
        SQRT((n * sum_premium_squared - n * avg_premium * avg_premium) * 
             (n * sum_deductible_squared - n * avg_deductible * avg_deductible))) AS premium_deductible_correlation,
    ((n * sum_premium_moop - n * avg_premium * avg_moop) / 
        SQRT((n * sum_premium_squared - n * avg_premium * avg_premium) * 
             (n * sum_moop_squared - n * avg_moop * avg_moop))) AS premium_moop_correlation
FROM CostCorrelation
ORDER BY plan_year, plan_type;

"""
# Execute and display results for Diagnostic Analytics
diagnostic_results = pd.read_sql(diagnostic_query, con=engine)
print("Diagnostic Analytics Results:")
print(diagnostic_results)

# Close the connection when done
engine.dispose()


In [ ]:
Insights:

    High Correlation between Premiums and MOOPs: 
        
                For both HMO and PPO plans, there is consistently a high correlation between 
                premiums and MOOPs. This indicates that plans with higher premiums tend to 
                also have higher maximum out-of-pocket costs. This pattern is particularly 
                strong in the PPO plans where correlations are near or at 0.995 in the 
                available years.
                
    Variable Correlation between Premiums and Deductibles: 
        
                The correlation between premiums and deductibles varies more than that between 
                premiums and MOOPs. For HMO plans, the correlation ranges from moderate to high 
                (around 0.706 to 0.967), suggesting that in some years, higher premiums are 
                associated with higher deductibles, but this relationship is not as consistent 
                as with MOOPs.
                
    Yearly Fluctuations: 
        
                There are fluctuations in the correlations from year to year, which could be influenced 
                by changes in plan offerings, regulatory changes, or shifts in the healthcare market.
        
Recommendations:
    
    For Consumers:
        
                When choosing plans, consider not just the premium but also the deductible and MOOP as 
                these costs are correlated. High premiums often come with high MOOPs, which could 
                significantly impact out-of-pocket expenses.
                
                Consumers might benefit from a detailed comparison of different plan types and years 
                to find the plan that best balances premiums with potential out-of-pocket costs.
                
    For Healthcare Providers and Insurers:
        
                Insurers should consider structuring their offerings to balance the correlation 
                between premiums and other costs, potentially offering more competitive and 
                attractive plans that could cater to a broader range of consumers.
                
                Healthcare providers could use this correlation data to advise patients or 
                clients, especially those managing chronic conditions, on selecting the 
                most cost-effective healthcare plans.
                
Prediction:
    
    Continued Correlation Trends: 
        
            Given the strong historical correlation between premiums and MOOPs, it is likely 
            that this trend will continue. Insurers may adjust deductibles and MOOPs in response 
            to market pressures or policy changes but expect the premium-MOOP relationship to remain 
            relatively stable.
        
    Market Adjustments: 
        
            If the correlation causes consumer dissatisfaction or if it becomes a focus of 
            regulatory review, insurers might adjust the structures of their plans. This could 
            lead to a future where premiums do not necessarily dictate MOOPs and deductibles 
            to the same extent, leading to more diversified plan offerings.